## 1. DATA ANALYST AGENT DATASETS

# Dataset 1: UCI Bank Marketing Dataset

In [26]:
import pandas as pd
import re
from ucimlrepo import fetch_ucirepo

# -----------------------------------
# LOAD DATA
# -----------------------------------
bank_marketing = fetch_ucirepo(id=222)

df = bank_marketing.data.features.copy()
df["target_term_deposit"] = bank_marketing.data.targets   # keep target with features


# -----------------------------------
# 1. CATEGORICAL CLEANING USING REGEX
# -----------------------------------
cat_cols = [
    "job", "marital", "education", "default",
    "housing", "loan", "contact", "poutcome",
    "day_of_week", "month"
]

def regex_clean(x):
    if pd.isna(x):
        return "unknown"

    x = str(x).lower()

    # remove unwanted punctuation using regex
    x = re.sub(r"[^a-z0-9\s\-\._]", "", x)   # keep alphanumerics, dash, underscore, dot

    # replace dash and dot with underscore using regex
    x = re.sub(r"[\-\.]", "_", x)

    # collapse multiple underscores
    x = re.sub(r"_+", "_", x)

    # strip whitespace/underscores
    x = x.strip("_ ").replace(" ", "_")

    # if empty, mark unknown
    return x if x else "unknown"

for col in cat_cols:
    df[col] = df[col].apply(regex_clean)


# -----------------------------------
# 2. HANDLE NUMERIC MISSING VALUES
# -----------------------------------
num_cols = ["age", "balance", "duration", "campaign", "pdays", "previous"]
df[num_cols] = df[num_cols].fillna(0)

# convert -1 in pdays to 0 (meaning "not contacted before")
df["pdays"] = df["pdays"].replace(-1, 0)


# -----------------------------------
# 3. CLEAN YES/NO FIELDS USING REGEX
# -----------------------------------
yn_cols = ["default", "housing", "loan", "target_term_deposit"]

for col in yn_cols:
    df[col] = df[col].astype(str).str.lower()
    df[col] = df[col].str.replace(r"[^a-z]", "", regex=True)  # remove unexpected chars
    df[col] = df[col].replace({"yes": "yes", "no": "no"})
    df[col] = df[col].apply(lambda x: "no" if x not in ["yes", "no"] else x)


# -----------------------------------
# 4. BUCKETIZE FINANCIAL COLUMNS (optional)
# -----------------------------------
def bucket_balance(x):
    if x < 0: return "negative"
    elif x < 500: return "low"
    elif x < 2500: return "medium"
    return "high"

df["balance_bucket"] = df["balance"].apply(bucket_balance)

def bucket_duration(x):
    if x < 100: return "short"
    elif x < 300: return "medium"
    return "long"

df["duration_bucket"] = df["duration"].apply(bucket_duration)


# -----------------------------------
# 5. NATURAL LANGUAGE PROFILE (optional)
# -----------------------------------
def build_profile(row):
    return (
        f"{row['age']}-year-old {row['job']} customer "
        f"who is {row['marital']} with {row['education']} education. "
        f"Balance: {row['balance_bucket']}. Housing: {row['housing']}, loan: {row['loan']}. "
        f"Contacted in {row['month']} ({row['day_of_week']}). "
        f"Call duration: {row['duration']} sec. "
        f"Previous outcome: {row['poutcome']}. "
        f"Subscribed: {row['target_term_deposit']}."
    )

df["customer_profile"] = df.apply(build_profile, axis=1)

df.to_csv('UCI Bank Marketing Dataset.csv)
df

,age,job,marital,education,default,balance,housing,loan,contact,day_of_week,month,duration,campaign,pdays,previous,poutcome,target_term_deposit,balance_bucket,duration_bucket,customer_profile
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,0,0,unknown,no,medium,medium,58-year-old management customer who is married...
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,0,0,unknown,no,low,medium,44-year-old technician customer who is single ...
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,0,0,unknown,no,low,short,33-year-old entrepreneur customer who is marri...
3,47,blue_collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,0,0,unknown,no,medium,short,47-year-old blue_collar customer who is marrie...
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,0,0,unknown,no,low,medium,33-year-old unknown customer who is single wit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,0,0,unknown,yes,medium,long,51-year-old technician customer who is married...
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,0,0,unknown,yes,medium,long,71-year-old retired customer who is divorced w...
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes,high,long,72-year-old retired customer who is married wi...
45209,57,blue_collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,0,0,unknown,no,medium,long,57-year-old blue_collar customer who is marrie...


# Dataset 2: UCI Adult Census Income Dataset

In [39]:
import pandas as pd
import re
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import MinMaxScaler

# -----------------------------------
# 1. LOAD DATA
# -----------------------------------
adult = fetch_ucirepo(id=2)

df = adult.data.features.copy()
df["income"] = adult.data.targets  # include target column

# -----------------------------------
# 2. HANDLE MISSING VALUES
# -----------------------------------
# In Adult dataset, missing values are represented by '?'
df.replace("?", "unknown", inplace=True)

# -----------------------------------
# 3. CLEAN CATEGORICAL VARIABLES USING REGEX
# -----------------------------------
cat_cols = [
    "workclass", "education", "marital-status",
    "occupation", "relationship", "race", "sex", "native-country"
]

def regex_clean(x):
    if pd.isna(x) or x == "unknown":
        return "unknown"
    x = str(x).lower()
    x = re.sub(r"[^a-z0-9\s\-\._]", "", x)  # remove unwanted chars
    x = re.sub(r"[\-\.]", "_", x)           # replace dash/dot with underscore
    x = re.sub(r"_+", "_", x)               # collapse multiple underscores
    x = x.strip("_ ").replace(" ", "_")     # clean edges and spaces
    return x if x else "unknown"

for col in cat_cols:
    df[col] = df[col].apply(regex_clean)

# -----------------------------------
# 4. SCALE SELECT NUMERIC FEATURES (leave age raw)
# -----------------------------------
num_cols_to_scale = ["fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]

scaler = MinMaxScaler()
df[num_cols_to_scale] = scaler.fit_transform(df[num_cols_to_scale])

# -----------------------------------
# 5. OPTIONAL: BUILD NATURAL-LANGUAGE PROFILE
# -----------------------------------
def build_profile(row):
    return (
        f"{row['age']}-year-old {row['sex']} {row['race']} individual "
        f"with {row['education']} education, working as {row['occupation']} "
        f"in {row['workclass']} sector. Marital status: {row['marital-status']}, "
        f"relationship: {row['relationship']}. "
        f"Capital gain: {row['capital-gain']:.2f}, capital loss: {row['capital-loss']:.2f}, "
        f"works {row['hours-per-week']:.2f} hours per week. "
        f"From {row['native-country']}. "
        f"Income target: {row['income']}."
    )

df["profile_text"] = df.apply(build_profile, axis=1)

# -----------------------------------
# 6. SAVE CLEANED DATA
# -----------------------------------
df.to_csv('UCI_Adult_Census_Income_Preprocessed.csv', index=False)

df


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,profile_text
0,39,state_gov,0.044131,bachelors,0.800000,never_married,adm_clerical,not_in_family,white,male,0.021740,0.0,0.397959,united_states,<=50K,39-year-old male white individual with bachelo...
1,50,self_emp_not_inc,0.048052,bachelors,0.800000,married_civ_spouse,exec_managerial,husband,white,male,0.000000,0.0,0.122449,united_states,<=50K,50-year-old male white individual with bachelo...
2,38,private,0.137581,hs_grad,0.533333,divorced,handlers_cleaners,not_in_family,white,male,0.000000,0.0,0.397959,united_states,<=50K,38-year-old male white individual with hs_grad...
3,53,private,0.150486,11th,0.400000,married_civ_spouse,handlers_cleaners,husband,black,male,0.000000,0.0,0.397959,united_states,<=50K,53-year-old male black individual with 11th ed...
4,28,private,0.220635,bachelors,0.800000,married_civ_spouse,prof_specialty,wife,black,female,0.000000,0.0,0.397959,cuba,<=50K,28-year-old female black individual with bache...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,private,0.137428,bachelors,0.800000,divorced,prof_specialty,not_in_family,white,female,0.000000,0.0,0.357143,united_states,<=50K.,39-year-old female white individual with bache...
48838,64,unknown,0.209130,hs_grad,0.533333,widowed,unknown,other_relative,black,male,0.000000,0.0,0.397959,united_states,<=50K.,64-year-old male black individual with hs_grad...
48839,38,private,0.245379,bachelors,0.800000,married_civ_spouse,prof_specialty,husband,white,male,0.000000,0.0,0.500000,united_states,<=50K.,38-year-old male white individual with bachelo...
48840,44,private,0.048444,bachelors,0.800000,divorced,adm_clerical,own_child,asian_pac_islander,male,0.054551,0.0,0.397959,united_states,<=50K.,44-year-old male asian_pac_islander individual...


# Dataset 3: Kaggle Credit Card Fraud Detection Dataset

In [41]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 1. LOAD DATA
df = pd.read_csv("creditcard.csv")

# 2. RENAME COLUMNS
new_col_names = {
    'Time': 'transaction_time',
    'Amount': 'transaction_amount',
    'Class': 'fraud_label'
}

for i in range(1, 29):
    new_col_names[f'V{i}'] = f'feature_{i}'

df.rename(columns=new_col_names, inplace=True)

# 3. CONVERT TARGET
df['fraud_label'] = df['fraud_label'].apply(lambda x: 'fraud' if x == 1 else 'non-fraud')

# 4. MIN-MAX SCALE PCA FEATURES TO [0,1]
feature_cols = [f'feature_{i}' for i in range(1, 29)]
scaler = MinMaxScaler()
df[feature_cols] = scaler.fit_transform(df[feature_cols])

# Optionally scale transaction_amount too
df['transaction_amount'] = MinMaxScaler().fit_transform(df[['transaction_amount']])

# 5. OPTIONAL: NATURAL-LANGUAGE SUMMARY
def build_profile(row):
    features = ', '.join([f"{row[f]:.3f}" for f in feature_cols])
    return (
        f"Transaction at {row['transaction_time']} seconds since start, "
        f"amount {row['transaction_amount']:.2f} (normalized). "
        f"Features: {features}. "
        f"Fraud label: {row['fraud_label']}."
    )

df['transaction_profile'] = df.apply(build_profile, axis=1)

# 6. OPTIONAL: STRATIFIED SAMPLING FOR BALANCED DATA
fraud_df = df[df['fraud_label'] == 'fraud']
non_fraud_df = df[df['fraud_label'] == 'non-fraud'].sample(n=len(fraud_df), random_state=42)
df_sample = pd.concat([fraud_df, non_fraud_df]).reset_index(drop=True)



df_sample.to_csv('Kaggle Credit Card Fraud Detection Dataset.csv')
df_sample

,transaction_time,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,transaction_amount,fraud_label,transaction_profile
0,406.0,0.919012,0.787855,0.809517,0.429154,0.762201,0.248677,0.249897,0.800314,0.367355,...,0.508396,0.658525,0.425381,0.580406,0.454498,0.421331,0.310216,0.000000,fraud,"Transaction at 406.0 seconds since start, amou..."
1,472.0,0.906588,0.733944,0.856275,0.353384,0.774870,0.252314,0.267339,0.784658,0.453446,...,0.530346,0.685868,0.342644,0.593612,0.401704,0.411845,0.313850,0.020591,fraud,"Transaction at 472.0 seconds since start, amou..."
2,4462.0,0.919163,0.785821,0.831180,0.355228,0.760185,0.262258,0.268781,0.781104,0.454573,...,0.466535,0.667999,0.370467,0.569143,0.336811,0.417241,0.310018,0.009339,fraud,"Transaction at 4462.0 seconds since start, amo..."
3,6986.0,0.883578,0.781591,0.792483,0.370723,0.758122,0.245862,0.244056,0.782717,0.454245,...,0.518287,0.658956,0.375025,0.592075,0.318049,0.401244,0.330364,0.002297,fraud,"Transaction at 6986.0 seconds since start, amo..."
4,7519.0,0.979262,0.799121,0.762821,0.461731,0.790114,0.249369,0.275794,0.780061,0.418588,...,0.477181,0.655679,0.162235,0.661482,0.518034,0.416326,0.316103,0.000039,fraud,"Transaction at 7519.0 seconds since start, amo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,76376.0,0.932369,0.775660,0.857731,0.209946,0.766235,0.246374,0.269442,0.785939,0.432222,...,0.506651,0.659535,0.493484,0.600636,0.576360,0.411670,0.310417,0.000895,non-fraud,"Transaction at 76376.0 seconds since start, am..."
980,60348.0,0.946303,0.782221,0.854634,0.292210,0.764564,0.257320,0.267594,0.789911,0.437408,...,0.540624,0.664711,0.438161,0.561757,0.375551,0.422253,0.316410,0.000109,non-fraud,"Transaction at 60348.0 seconds since start, am..."
981,47151.0,0.975201,0.765699,0.835526,0.303727,0.767936,0.274905,0.263430,0.788851,0.481417,...,0.510296,0.659926,0.195473,0.619785,0.394194,0.417811,0.313965,0.003850,non-fraud,"Transaction at 47151.0 seconds since start, am..."
982,53857.0,0.937946,0.800673,0.777166,0.288225,0.774539,0.258545,0.267696,0.790726,0.481826,...,0.497670,0.666776,0.154349,0.579755,0.378800,0.430398,0.320137,0.000035,non-fraud,"Transaction at 53857.0 seconds since start, am..."


# Dataset 4: Consumer Expenditure Survey (BLS - US Bureau of Labor Statistics)

In [2]:
import pandas as pd
import glob
import os
import sys
from functools import reduce

# --- Configuration ---
# The primary keys used for merging the different CEX record types.
# NEWID is the unique consumer unit ID. REFYR/REFMO specifies the interview year/month.
MERGE_KEYS = 'NEWID'

# Find all CSV files recursively in the current directory and all subdirectories.
# This ensures we catch files regardless of which subfolder they are in.
try:
    ALL_FILES = glob.glob('**/*.csv', recursive=True)
    if not ALL_FILES:
        print("Error: No CSV files found. Ensure the script is running from inside the main data folder.")
        sys.exit(1)
except Exception as e:
    print(f"An error occurred during file search: {e}")
    sys.exit(1)

def load_and_combine_data(file_prefix):
    """
    Finds all CSV files matching a prefix (e.g., 'fmli'), loads them,
    and combines them into a single DataFrame.
    """
    print(f"\n--- Processing {file_prefix.upper()} files ---")
    
    # Filter the master list of ALL_FILES based on the file prefix
    files_to_process = [f for f in ALL_FILES if os.path.basename(f).startswith(file_prefix)]
    
    if not files_to_process:
        print(f"No files found for prefix: {file_prefix}. Skipping combination for this type.")
        return None

    df_list = []
    
    for file in files_to_process:
        try:
            # low_memory=False is crucial for CEX data due to inconsistent column data types
            # Only load necessary columns for ITBI/ITII to avoid memory overflow on large transaction files
            if file_prefix in ['itbi', 'itii']:
                 # We only need the keys, the UCC code (the type of item), and the VALUE amount.
                 df = pd.read_csv(file, low_memory=False, usecols=['NEWID', 'REFYR', 'REFMO', 'UCC', 'VALUE'])
            else:
                df = pd.read_csv(file, low_memory=False)

            df_list.append(df)
            print(f"Successfully loaded: {file} ({len(df):,} rows)")
        except Exception as e:
            print(f"Error loading {file}: {e}. Skipping file. Details: {e}")
            
    if df_list:
        combined_df = pd.concat(df_list, ignore_index=True)
        print(f"Combined {len(df_list)} files into {len(combined_df):,} total rows for {file_prefix.upper()}")
        
        # Standardize merge keys to string type to prevent merge errors
        for col in MERGE_KEYS:
            if col in combined_df.columns:
                combined_df[col] = combined_df[col].astype(str)
        
        return combined_df
    return None

def aggregate_transaction_data(df, prefix):
    """
    Aggregates transaction-level data (ITBI or ITII) by summing the 'VALUE' 
    for each unique NEWID/REFYR/REFMO and UCC code.
    
    This turns millions of transaction rows into a condensed table 
    that can be merged without MemoryError.
    """
    if df is None:
        return None
    
    print(f"\n--- Aggregating {prefix.upper()} Data ({len(df):,} rows) ---")
    
    # Use pivot_table to quickly aggregate:
    # Rows: Unique Consumer Unit (defined by MERGE_KEYS)
    # Columns: Unique UCC codes (each UCC is a spending/income category)
    # Values: Sum of the VALUE (dollar amount)
    try:
        agg_df = df.pivot_table(
            index=MERGE_KEYS,
            columns='UCC',
            values='VALUE',
            aggfunc='sum',
            fill_value=0 # Fill missing transactions with 0
        ).reset_index()
    except Exception as e:
        # Fallback for extreme memory pressure, though pivot_table is usually the best approach
        print(f"Fatal Error during pivot_table aggregation: {e}")
        print("Please consider running this script on a machine with more RAM.")
        sys.exit(1)

    
    # --- FIX for TypeError: 'in <string>' requires string as left operand, not int ---
    # Convert all column names to strings before comparison/renaming
    agg_df.columns = agg_df.columns.astype(str)
    
    # Rename columns to identify source (e.g., UCC_000100 becomes ITBI_000100)
    agg_df.columns = [f'{prefix.upper()}_{col}' if col not in MERGE_KEYS else col for col in agg_df.columns]
    
    print(f"Aggregated {prefix.upper()} data reduced to {len(agg_df):,} rows (one row per consumer unit/month).")
    return agg_df

def preprocess_and_merge(fmli_df, itbi_agg_df, itii_agg_df):
    """
    Merges the FMLI base with the aggregated transaction data.
    """
    print("\n--- Starting Merging of Aggregated Data (Left Join on FMLI) ---")

    # List of DataFrames to merge, starting with the base (FMLI)
    dfs_to_merge = [itbi_agg_df, itii_agg_df]
    
    final_cex_df = fmli_df
    
    for df in dfs_to_merge:
        if df is not None:
            # We use a Left Join to ensure every Consumer Unit (from FMLI) is kept
            final_cex_df = pd.merge(
                final_cex_df,
                df,
                on=MERGE_KEYS,
                how='left'
            )
            print(f"Current DataFrame size after merge: {len(final_cex_df):,} rows.")
        else:
            print(f"Skipping merge (DF is None).")

    # Fill NaN values (where a Consumer Unit had no transactions for a merged UCC code) with 0
    final_cex_df = final_cex_df.fillna(0) 

    return final_cex_df

# --- Main Execution ---

# Step 1: Combine files by type
fmli_df = load_and_combine_data('fmli')
itbi_df = load_and_combine_data('itbi')
itii_df = load_and_combine_data('itii')

# Step 2: Ensure the core demographic file (fmli) is available to proceed
if fmli_df is None:
    print("\nFatal Error: The core FMLI (Family/Demographics) file(s) could not be loaded. Cannot proceed with merging.")
    sys.exit(1)

# Step 3: Aggregate transaction files (ITBI and ITII) to prevent MemoryError
itbi_agg_df = aggregate_transaction_data(itbi_df, 'itbi')
itii_agg_df = aggregate_transaction_data(itii_df, 'itii')


# Step 4: Merge the combined and aggregated DataFrames
cex_master_df = preprocess_and_merge(fmli_df, itbi_agg_df, itii_agg_df)

# Step 5: Final inspection and saving
print("\n--- Final Dataset Summary ---")
print(f"Master CEX DataFrame shape: {cex_master_df.shape}")
print("First 5 rows of the Master DataFrame:")
print(cex_master_df.head())

# Save the final dataset to a single CSV file, ready for RAG indexing
output_filename = 'cex_master_dataset_for_rag.csv'
cex_master_df.to_csv(output_filename, index=False)
print(f"\nSuccessfully saved the complete dataset to: {output_filename}")

# --- Handling Other Files (Identification) ---
# This part helps you identify the other files you mentioned
processed_files = []
for prefix in ['fmli', 'itbi', 'itii']:
    processed_files.extend([f for f in ALL_FILES if os.path.basename(f).startswith(prefix)])

# Find files in ALL_FILES that were NOT in the processed list
unprocessed_files = [f for f in ALL_FILES if f not in processed_files]

if unprocessed_files:
    print("\n--- NOTE ON UNPROCESSED FILES ---")
    print("The following files were found in your folders but NOT combined or merged, as they do not match the core CEX patterns ('fmli', 'itbi', 'itii'):")
    for f in unprocessed_files:
        print(f"- {f}")
    print("\nThese are likely supplementary files (e.g., codebooks, lookup tables) and may need custom loading/merging depending on your RAG strategy.")


--- Processing FMLI files ---
Successfully loaded: intrvw23\intrvw23\fmli232.csv (4,751 rows)
Successfully loaded: intrvw23\intrvw23\fmli233.csv (4,770 rows)
Successfully loaded: intrvw23\intrvw23\fmli234.csv (4,662 rows)
Successfully loaded: intrvw23\intrvw23\fmli241.csv (4,688 rows)
Combined 4 files into 18,871 total rows for FMLI

--- Processing ITBI files ---
Successfully loaded: intrvw23\intrvw23\itbi232.csv (271,662 rows)
Successfully loaded: intrvw23\intrvw23\itbi233.csv (272,544 rows)
Successfully loaded: intrvw23\intrvw23\itbi234.csv (265,401 rows)
Successfully loaded: intrvw23\intrvw23\itbi241.csv (268,224 rows)
Combined 4 files into 1,077,831 total rows for ITBI

--- Processing ITII files ---
Successfully loaded: intrvw23\intrvw23\itii232.csv (330,450 rows)
Successfully loaded: intrvw23\intrvw23\itii233.csv (330,840 rows)
Successfully loaded: intrvw23\intrvw23\itii234.csv (322,320 rows)
Successfully loaded: intrvw23\intrvw23\itii241.csv (325,200 rows)
Combined 4 files into 

In [ ]:
import pandas as pd
import os
import sys

# --- Configuration ---
INPUT_FILENAME = 'cex_master_dataset_for_rag.csv'
OUTPUT_FILENAME = 'cex_master_clean.csv'

# --- 1. Define the Column Renaming Map ---
# Map the original CEX codes (FMLI/ITBI/ITII) to descriptive, RAG-friendly names.
RENAMING_MAP = {
    # --- Identifiers & Weight ---
    'NEWID': 'Consumer_Unit_ID',
    'REFYR': 'Reference_Year',
    'REFMO': 'Reference_Month',
    'FINLWT21': 'Final_Interview_Weight_Factor',
    
    # --- Demographics (FMLI) ---
    'FAM_SIZE': 'Family_Size',
    'AGE_REF': 'Age_Reference_Person',
    'EDUC_REF': 'Education_Reference_Person',
    'CUTENURE': 'Homeownership_Status', # 1=Owned, 2=Rented, 3=Occupied without payment
    'REGION': 'Geographic_Region', # 1=NE, 2=MW, 3=S, 4=W
    'BLS_URBN': 'BLS_Urban_Rural_Status',
    'WORKCOMP': 'Employment_Status_Reference_Person',
    
    # --- Total Income & Spending (FMLI) ---
    'FINCBTAX': 'Total_Income_Before_Tax',
    'FSALARYX': 'Total_Salary_Wage_Income',
    'FSSIX': 'Total_Social_Security_Income',
    
    # --- Aggregated Expenditure Categories (ITBI_) ---
    # NOTE: You must consult the CEX UCC Codebook for the full list.
    # We include key examples here:
    'ITBI_005100': 'EXP_Food_At_Home',
    'ITBI_005110': 'EXP_Food_Away_From_Home',
    'ITBI_900000': 'EXP_Shelter_Rent_or_Mortgage',
    'ITBI_480500': 'EXP_Gasoline_And_Motor_Oil',
    'ITBI_550000': 'EXP_Vehicle_Purchases_Net',
    'ITBI_210000': 'EXP_Apparel_and_Services',
    'ITBI_320100': 'EXP_Health_Insurance_Premiums',
    
    # --- Aggregated Income Categories (ITII_) ---
    'ITII_900030': 'INC_Wages_And_Salaries',
    'ITII_900140': 'INC_Social_Security',
    'ITII_900150': 'INC_Public_Assistance_Welfare',
}

def rename_and_save_data():
    """
    Loads the merged CEX file, renames columns, and saves the cleaned file.
    """
    if not os.path.exists(INPUT_FILENAME):
        print(f"Error: Input file '{INPUT_FILENAME}' not found.")
        print("Please ensure you run the 'data_integration.py' script successfully first.")
        sys.exit(1)

    print(f"Loading merged dataset: {INPUT_FILENAME}...")
    try:
        # Load the large file
        df = pd.read_csv(INPUT_FILENAME, low_memory=False)
    except Exception as e:
        print(f"Failed to load the CSV: {e}")
        sys.exit(1)

    print(f"DataFrame loaded with {len(df.columns)} columns.")

    # --- 2. Rename Columns ---
    # Filter the renaming map to only include columns that actually exist in the DataFrame
    valid_renaming_map = {
        old_name: new_name 
        for old_name, new_name in RENAMING_MAP.items() 
        if old_name in df.columns
    }
    
    # Identify and apply a generalized rename to the remaining UCC codes
    # This prevents hundreds of unreadable UCC codes from remaining.
    generic_rename_map = {}
    for col in df.columns:
        if col.startswith('ITBI_') and col not in valid_renaming_map:
            generic_rename_map[col] = f'EXP_UCC_{col[5:]}'
        elif col.startswith('ITII_') and col not in valid_renaming_map:
            generic_rename_map[col] = f'INC_UCC_{col[5:]}'

    final_renaming_map = {**valid_renaming_map, **generic_rename_map}
    
    # Apply the renaming
    df.rename(columns=final_renaming_map, inplace=True)
    
    print(f"Successfully renamed {len(final_renaming_map)} columns.")
    
    # --- 3. Save the Cleaned File ---
    print(f"Saving cleaned dataset to: {OUTPUT_FILENAME}...")
    df.to_csv(OUTPUT_FILENAME, index=False)
    print(f"Success! Cleaned dataset saved. Total columns: {len(df.columns)}.")


if __name__ == '__main__':
    rename_and_save_data()

#  Dataset 5: FRED Economic Data (Federal Reserve

In [4]:
# ================================
# FRED Macro Data Preprocessing
# ================================

from fredapi import Fred
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

# -------------------------
# 1. Initialize FRED API
# -------------------------
api_key = '7242fe69a51188cad3114480ea35cf15'
fred = Fred(api_key=api_key)

# -------------------------
# 2. Define series mapping
# -------------------------
series_mapping = {
    'CPIAUCNS': 'Consumer_Price_Index',         # CPI, All Urban Consumers
    'UNRATE': 'Unemployment_Rate',             # Unemployment Rate
    'FEDFUNDS': 'Federal_Funds_Rate',          # Federal Funds Rate
    'W875RX1': 'Personal_Income',              # Personal Income
    'UMCSENT': 'Consumer_Confidence_Index',    # Michigan Consumer Sentiment
    'CSUSHPISA': 'House_Price_Index'           # S&P/Case-Shiller House Price Index
}

# -------------------------
# 3. Fetch series
# -------------------------
data = {}
for series_id, friendly_name in series_mapping.items():
    try:
        data[friendly_name] = fred.get_series(series_id)
    except Exception as e:
        print(f"Error fetching {series_id}: {e}")

df = pd.DataFrame(data)
df.index.name = 'Date'

# Save raw dataset
df.to_csv('fred_macro_data_raw.csv')

# -------------------------
# 4. Handle NaN values
# -------------------------
# Forward fill then backward fill
df_clean = df.fillna(method='ffill').fillna(method='bfill')

# -------------------------
# 5. Stationarity check
# -------------------------
non_stationary_cols = []

print("\n=== Stationarity Check (ADF Test) ===")
for col in df_clean.columns:
    result = adfuller(df_clean[col].dropna())
    p_value = result[1]
    if p_value >= 0.05:
        non_stationary_cols.append(col)
    print(f"{col}: ADF Statistic={result[0]:.3f}, p-value={p_value:.3f}")

# -------------------------
# 6. Transform non-stationary series
# -------------------------
# Use log difference (log-return) to make stationary
df_stationary = df_clean.copy()
for col in non_stationary_cols:
    # Avoid log(0)
    df_stationary[col] = np.log(df_clean[col].replace(0, np.nan)).diff()

df_stationary = df_stationary.dropna()

# -------------------------
# 7. Seasonal adjustment (optional example for CPI)
# -------------------------
if 'Consumer_Price_Index' in df_stationary.columns:
    result = seasonal_decompose(df_clean['Consumer_Price_Index'], model='additive', period=12)
    df_stationary['CPI_seasonally_adjusted'] = df_clean['Consumer_Price_Index'] - result.seasonal

# -------------------------
# 8. Save processed dataset
# -------------------------
df_stationary.to_csv('fred_macro_data_processed.csv')

print("\n✅ Preprocessing complete!")
print("Raw data saved to: fred_macro_data_raw.csv")
print("Processed data saved to: fred_macro_data_processed.csv")

C:\Users\itsam\AppData\Local\Temp\ipykernel_10924\3335750587.py:49: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_clean = df.fillna(method='ffill').fillna(method='bfill')



=== Stationarity Check (ADF Test) ===
Consumer_Price_Index: ADF Statistic=3.197, p-value=1.000
Unemployment_Rate: ADF Statistic=-3.587, p-value=0.006
Federal_Funds_Rate: ADF Statistic=-2.295, p-value=0.174
Personal_Income: ADF Statistic=5.504, p-value=1.000
Consumer_Confidence_Index: ADF Statistic=-3.849, p-value=0.002
House_Price_Index: ADF Statistic=2.482, p-value=0.999

✅ Preprocessing complete!
Raw data saved to: fred_macro_data_raw.csv
Processed data saved to: fred_macro_data_processed.csv


# Dataset 6: World Bank Open Data

In [13]:
import wbdata
import pandas as pd
import datetime

# -------------------------
# 1. Define indicators
# -------------------------
indicators = {
    'NY.GDP.MKTP.CD': 'GDP',                   # GDP (current US$)
    'SP.POP.TOTL': 'Population',               # Total population
    'SP.DYN.LE00.IN': 'Life_Expectancy',       # Life expectancy at birth
    'SI.POV.DDAY': 'Poverty_Rate'              # Poverty headcount ratio (%)
}

# -------------------------
# 2. Get all country ISO codes
# -------------------------
all_countries = wbdata.get_countries()  # returns list of dicts
countries = [c['id'] for c in all_countries if c['region']['id'] != 'NA']  # exclude aggregates

# -------------------------
# 3. Fetch data for all countries (no data_date)
# -------------------------
df = wbdata.get_dataframe(
    indicators,
    country=countries
)

# -------------------------
# 4. Reset index
# -------------------------
df.reset_index(inplace=True)

# -------------------------
# 5. Filter by date range
# -------------------------
start_year = 1960
end_year = 2023
df['date'] = pd.to_datetime(df['date'])
df = df[(df['date'].dt.year >= start_year) & (df['date'].dt.year <= end_year)]

# -------------------------
# 6. Preview and save
# -------------------------
print(df.head())
df.to_csv('worldbank_data.csv', index=False)


  country       date           GDP  Population  Life_Expectancy  Poverty_Rate
1   Aruba 2023-01-01  3.648573e+09    107359.0           76.353           NaN
2   Aruba 2022-01-01  3.279344e+09    107310.0           76.226           NaN
3   Aruba 2021-01-01  2.929447e+09    107700.0           73.655           NaN
4   Aruba 2020-01-01  2.481857e+09    108587.0           75.406           NaN
5   Aruba 2019-01-01  3.395799e+09    109203.0           76.019           NaN


In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# -------------------------
# 1. Load CSV
# -------------------------
df = pd.read_csv('worldbank_data.csv')

# Preview
print(df.head())
print(df.info())

# -------------------------
# 2. Handle missing values
# -------------------------
# Option 1: Forward fill by country (use previous year)
df.sort_values(by=['country', 'date'], inplace=True)
df.fillna(method='ffill', inplace=True)

# Option 2: Backward fill by country
df.fillna(method='bfill', inplace=True)

# Option 3: If still missing, fill with overall mean
df.fillna(df.mean(numeric_only=True), inplace=True)

# -------------------------
# 3. Handle different reporting standards
# -------------------------
# Example: Convert all GDP and income to USD if needed
# For now, assume GDP is in current USD, so no conversion required
# Otherwise, you would multiply by conversion rates for historical years

# -------------------------
# 4. Normalize numeric columns
# -------------------------
numeric_cols = df.select_dtypes(include=[np.number]).columns
scaler = MinMaxScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# -------------------------
# 5. Preview processed data
# -------------------------
print(df.head())

# -------------------------
# 6. Save processed CSV
# -------------------------
df.to_csv('worldbank_data_processed.csv', index=False)
print("✅ Preprocessing complete. Saved to worldbank_data_processed.csv")


  country        date           GDP  Population  Life_Expectancy  Poverty_Rate
0   Aruba  2023-01-01  3.648573e+09    107359.0           76.353           NaN
1   Aruba  2022-01-01  3.279344e+09    107310.0           76.226           NaN
2   Aruba  2021-01-01  2.929447e+09    107700.0           73.655           NaN
3   Aruba  2020-01-01  2.481857e+09    108587.0           75.406           NaN
4   Aruba  2019-01-01  3.395799e+09    109203.0           76.019           NaN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13888 entries, 0 to 13887
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   country          13888 non-null  object 
 1   date             13888 non-null  object 
 2   GDP              11366 non-null  float64
 3   Population       13858 non-null  float64
 4   Life_Expectancy  13854 non-null  float64
 5   Poverty_Rate     2389 non-null   float64
dtypes: float64(4), object(2)
memory usage: 651.1+

C:\Users\itsam\AppData\Local\Temp\ipykernel_10924\14183729.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\itsam\AppData\Local\Temp\ipykernel_10924\14183729.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


## 2. INVESTMENT ADVISOR AGENT DATASETS

# Dataset 1: S&P 500 & NASDAQ Historical OHLCV Data (Kaggle

In [29]:
import pandas as pd
import glob
import os
from sklearn.preprocessing import MinMaxScaler

# -------------------------
# 1. Folder path & CSV files
# -------------------------
folder_path = r"C:\Users\itsam\Downloads\archive"
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

if not csv_files:
    raise ValueError("No CSV files found in the folder. Check the path!")

# -------------------------
# 2. Load and combine all CSVs
# -------------------------
df_list = []
for file in csv_files:
    df = pd.read_csv(file, parse_dates=['date'])
    df_list.append(df)

data = pd.concat(df_list, ignore_index=True)

# -------------------------
# 3. Rename columns for clarity
# -------------------------
rename_mapping = {
    'date': 'Date',
    'open': 'Open_Price',
    'high': 'High_Price',
    'low': 'Low_Price',
    'close': 'Close_Price',
    'adjusted_close': 'Adj_Close',
    'volume': 'Volume',
    'change_percent': 'Change_Pct',
    'avg_vol_20d': 'Avg_Vol_20D',
    'raw_close': 'Raw_Close'
}

existing_cols = {k: v for k, v in rename_mapping.items() if k in data.columns}
data.rename(columns=existing_cols, inplace=True)

# -------------------------
# 4. Handle missing values
# -------------------------
numeric_cols = [col for col in ['Open_Price', 'High_Price', 'Low_Price',
                                'Close_Price', 'Adj_Close', 'Volume',
                                'Change_Pct', 'Avg_Vol_20D', 'Raw_Close'] 
                if col in data.columns]

# Forward fill then backward fill
data[numeric_cols] = data[numeric_cols].fillna(method='ffill').fillna(method='bfill')

# Optional: If raw_close is still NaN, fill from Close_Price
if 'Raw_Close' in data.columns:
    data['Raw_Close'] = data['Raw_Close'].fillna(data['Close_Price'])

# -------------------------
# 5. Normalize numeric columns
# -------------------------
scaler = MinMaxScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

# -------------------------
# 6. Save final cleaned dataset
# -------------------------
output_file = os.path.join(folder_path, "S&P500_NASDAQ_Cleaned.csv")
data.to_csv(output_file, index=False)

print(f"Data cleaned and saved to: {output_file}")
data.head()

C:\Users\itsam\AppData\Local\Temp\ipykernel_10924\926265880.py:53: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[numeric_cols] = data[numeric_cols].fillna(method='ffill').fillna(method='bfill')


Data cleaned and saved to: C:\Users\itsam\Downloads\archive\S&P500_NASDAQ_Cleaned.csv


,Date,Open_Price,High_Price,Low_Price,Close_Price,Volume,Change_Pct,Avg_Vol_20D,Adj_Close,Raw_Close
0,1985-09-25,0.005792,0.005784,0.005831,0.005824,0.000000,0.509429,0.015504,0.052333,0.192496
1,1985-09-26,0.005763,0.005755,0.005801,0.005794,0.000000,0.509429,0.015504,0.052333,0.192496
2,1985-09-27,0.005763,0.005755,0.005801,0.005794,0.000000,0.521662,0.015504,0.052333,0.192496
3,1985-09-30,0.005762,0.005754,0.005800,0.005793,0.000000,0.521152,0.015504,0.052333,0.192496
4,1985-10-01,0.005762,0.005837,0.005797,0.005876,0.013179,0.556575,0.015504,0.052333,0.192496


#  Dataset 2: Yahoo Finance API (via yfinance Python Library

In [33]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm

# -------------------------
# 1. Define tickers
# -------------------------
# Example: S&P 500 ETFs + index tickers
tickers = ["AAPL", "MSFT", "GOOG", "SPY", "QQQ"]  # add more as needed

# -------------------------
# 2. Download data
# -------------------------
all_data = []

for ticker_symbol in tqdm(tickers, desc="Downloading tickers"):
    try:
        ticker = yf.Ticker(ticker_symbol)
        
        # Historical price + OHLCV data
        hist = ticker.history(period="max")  # fetch maximum available
        
        if hist.empty:
            print(f"Warning: No data for {ticker_symbol}")
            continue

        # Reset index to make 'Date' a column
        hist = hist.reset_index()
        
        # Add ticker column
        hist['Ticker'] = ticker_symbol
        
        # Include dividends and splits as columns
        hist['Dividends'] = hist['Dividends'].fillna(0)
        hist['Stock_Splits'] = hist['Stock Splits'].fillna(0)
        
        # Append to list
        all_data.append(hist)
        
    except Exception as e:
        print(f"Error fetching {ticker_symbol}: {e}")

# -------------------------
# 3. Concatenate all tickers
# -------------------------
if all_data:
    df_all = pd.concat(all_data, axis=0, ignore_index=True)
    
    # Optional: sort by date and ticker
    df_all = df_all.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)
    
    # -------------------------
    # 4. Save to CSV
    # -------------------------
    df_all.to_csv("yfinance_all_tickers.csv", index=False)
    print("Saved concatenated data to 'yfinance_all_tickers.csv'")
else:
    print("No data downloaded.")

Saved concatenated data to 'yfinance_all_tickers.csv'


In [35]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# --- Configuration ---
FILE_PATH = 'yfinance_all_tickers.csv'
OUTPUT_FILE_PATH = 'preprocessed_stock_data.csv'
# Define the threshold for considering a stock "inactive" (e.g., hasn't traded in the last 6 months)
INACTIVE_THRESHOLD = 180 # days

def load_and_preprocess_data(file_path):
    """
    Loads historical stock data and performs essential preprocessing steps,
    including handling missing values.
    """
    print(f"Loading data from {file_path}...")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

    # 1. Initial Cleaning and Type Conversion
    
    # Ensure Date is in datetime format and set as index
    # Note: The original Date column in the CSV may include timezone info, which we handle
    df['Date'] = pd.to_datetime(df['Date'], utc=True).dt.tz_localize(None)
    
    # Convert core columns to numeric, coercing errors (useful for handling 'N/A' or bad data)
    numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Capital Gains']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
    # Identify categorical columns (primarily 'Ticker')
    categorical_cols = ['Ticker']

    # Drop rows where 'Ticker' is missing, as we cannot identify the security
    df.dropna(subset=categorical_cols, inplace=True)
    
    # --- NEW IMPUTATION STEP ---
    print("\nStarting Missing Value Imputation...")
    
    # Impute Numerical Columns: Forward Fill (FFill) is the best choice for time-series data, 
    # as we assume the price or volume from the previous day carries over if the current day's data is missing.
    # Group by Ticker before applying FFill to ensure we don't use data from a different stock to fill gaps.
    df = df.sort_values(by=['Ticker', 'Date']).set_index('Date')
    df[numeric_cols] = df.groupby('Ticker')[numeric_cols].ffill()
    df.reset_index(inplace=True)
    
    # After FFill, if the first few rows of a stock still have NaNs (no previous data to fill from),
    # we can fill them with 0 for Dividends/Splits/Capital Gains, as a missing value implies 0 for these events.
    # For price/volume, we will drop the remaining rows, as FFill should handle most cases.
    df.dropna(subset=['Close'], inplace=True)
    
    # Since Dividends, Stock Splits, and Capital Gains should be 0 if missing, fill remaining NaNs with 0
    df[['Dividends', 'Stock Splits', 'Capital Gains']] = df[['Dividends', 'Stock Splits', 'Capital Gains']].fillna(0)
    
    print(f"Shape after imputation and initial cleaning: {df.shape}")
    print(f"Number of unique tickers: {df['Ticker'].nunique()}")
    
    # 2. Check and Account for Stock Splits (Already handled by data source, but audit remains)
    split_count = df[df['Stock Splits'] != 0].shape[0]
    print(f"\nAudit: Found {split_count} historical dates with recorded Stock Splits.")
    
    # 3. Handle Delisted/Inactive Stocks
    
    latest_data_date = df['Date'].max()
    print(f"Latest trading date in dataset: {latest_data_date.date()}")
    
    # Calculate the last recorded date for each ticker
    last_dates = df.groupby('Ticker')['Date'].max()
    
    # Identify tickers whose last date is older than the INACTIVE_THRESHOLD
    inactive_cutoff_date = latest_data_date - timedelta(days=INACTIVE_THRESHOLD)
    inactive_tickers = last_dates[last_dates < inactive_cutoff_date].index.tolist()
    
    print(f"\nIdentified {len(inactive_tickers)} tickers as 'Inactive' (last trade before {inactive_cutoff_date.date()}):")
    # Display the first 10 inactive tickers for user inspection
    print(inactive_tickers[:10]) 

    # Filter out inactive tickers
    df_clean = df[~df['Ticker'].isin(inactive_tickers)].copy()
    print(f"\nShape after removing inactive/delisted stocks: {df_clean.shape}")
    
    # 4. Quality Filtering (Handling Micro-cap/Low-quality Data)
    
    # Remove rows with zero volume or non-positive close prices (often a sign of bad data)
    initial_rows = df_clean.shape[0]
    df_clean = df_clean[
        (df_clean['Volume'] > 0) & 
        (df_clean['Close'] > 0)
    ]
    rows_removed = initial_rows - df_clean.shape[0]
    print(f"Removed {rows_removed} low-quality rows (Volume=0 or Close<=0).")
    
    # 5. Final Output
    
    final_ticker_count = df_clean['Ticker'].nunique()
    print(f"\n--- Preprocessing Summary ---")
    print(f"Final dataset shape: {df_clean.shape}")
    print(f"Final number of unique, active tickers: {final_ticker_count}")
    print(f"Cleaned data saved to {OUTPUT_FILE_PATH}")
    
    # Save the cleaned data to a new CSV file
    df_clean.to_csv(OUTPUT_FILE_PATH, index=False)

    return df_clean

# Run the preprocessing script
if __name__ == "__main__":
    preprocessed_data = load_and_preprocess_data(FILE_PATH)
    
    # Optional: Display a snapshot of the cleaned data
    if preprocessed_data is not None:
        print("\n--- Snapshot of Preprocessed Data ---")
        print(preprocessed_data.head())

Loading data from yfinance_all_tickers.csv...

Starting Missing Value Imputation...
Shape after imputation and initial cleaning: (41658, 11)
Number of unique tickers: 5

Audit: Found 18 historical dates with recorded Stock Splits.
Latest trading date in dataset: 2025-11-20

Identified 0 tickers as 'Inactive' (last trade before 2025-05-24):
[]

Shape after removing inactive/delisted stocks: (41658, 11)
Removed 1 low-quality rows (Volume=0 or Close<=0).

--- Preprocessing Summary ---
Final dataset shape: (41657, 11)
Final number of unique, active tickers: 5
Cleaned data saved to preprocessed_stock_data.csv

--- Snapshot of Preprocessed Data ---
                 Date      Open      High       Low     Close     Volume  \
0 1980-12-12 05:00:00  0.098389  0.098817  0.098389  0.098389  469033600   
1 1980-12-15 05:00:00  0.093684  0.093684  0.093256  0.093256  175884800   
2 1980-12-16 05:00:00  0.086839  0.086839  0.086412  0.086412  105728000   
3 1980-12-17 05:00:00  0.088550  0.088978  0.

#  Dataset 3: Alpha Vantage Stock Market Data API

In [10]:
import time
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

# ---------------------
# CONFIGURATION
# ---------------------
# Each ticker gets its own API key
TICKER_API_KEYS = {
    "AAPL": "9VNGCY5YXNT0YZMJ",
    "MSFT": "UVAAAVPKVEHB7AGC",
    "GOOG": "63BQ32DSX4HMPB9X",
    "SPY": "A57406I9BSFDPI7H",
    "QQQ": "KHN50642ENXFIC21"
}

BASE_URL = "https://www.alphavantage.co/query"
SECONDS_PER_CALL = 12  # Rate limiting between calls


# ===========================
# Technical Indicator Functions
# ===========================

def calculate_sma(series, period=20):
    """Calculate Simple Moving Average"""
    return series.rolling(window=period).mean()


def calculate_rsi(series, period=14):
    """Calculate Relative Strength Index"""
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


def calculate_macd(series, fast=12, slow=26, signal=9):
    """Calculate MACD, Signal, and Histogram"""
    exp1 = series.ewm(span=fast, adjust=False).mean()
    exp2 = series.ewm(span=slow, adjust=False).mean()
    
    macd = exp1 - exp2
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    histogram = macd - signal_line
    
    return macd, signal_line, histogram


# ===========================
# API Call Function
# ===========================

def call_api(params, api_key, ticker):
    """Make API request with specific API key"""
    params["apikey"] = api_key
    
    key_display = f"{api_key[:4]}...{api_key[-4:]}"
    print(f"   🔑 Using API key: {key_display}")
    
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = requests.get(BASE_URL, params=params, timeout=30)

            if not response.text.strip():
                print(f"   ❗ Empty response (attempt {attempt + 1}/{max_retries})")
                time.sleep(5)
                continue

            data = response.json()

            print(f"   📋 Response keys: {list(data.keys())}")

            if "Note" in data:
                print(f"   ❗ Rate limit hit: {data['Note']}")
                return None
                
            if "Information" in data:
                print(f"   ❗ API message: {data['Information']}")
                return None

            if "Error Message" in data:
                print(f"   ❗ API Error: {data['Error Message']}")
                return None

            if "Time Series (Daily)" in data:
                print(f"   ✅ Successfully received data!")
            else:
                print(f"   ⚠️ Unexpected response format")
                print(f"   📄 Response preview: {str(data)[:300]}")

            return data

        except Exception as e:
            print(f"   ❗ Error: {e} (attempt {attempt + 1}/{max_retries})")
            time.sleep(5)

    print("   ❌ Failed after all retries")
    return None


# ===========================
# MAIN DOWNLOAD LOOP
# ===========================

all_data = []
successful_tickers = []
failed_tickers = []

print(f"\n{'='*60}")
print(f"📊 Alpha Vantage Multi-Key Stock Data Fetcher")
print(f"{'='*60}")
print(f"✅ {len(TICKER_API_KEYS)} tickers with {len(TICKER_API_KEYS)} dedicated API keys")
print(f"✅ Each ticker uses its own API key\n")

for i, (ticker, api_key) in enumerate(TICKER_API_KEYS.items(), 1):
    print(f"\n{'='*60}")
    print(f"[{i}/{len(TICKER_API_KEYS)}] Processing: {ticker}")
    print(f"{'='*60}")

    if i > 1:
        print(f"⏳ Waiting {SECONDS_PER_CALL} seconds...")
        time.sleep(SECONDS_PER_CALL)

    # *** FREE ENDPOINT ***
    params_daily = {
        "function": "TIME_SERIES_DAILY",
        "symbol": ticker,
        "outputsize": "full"
    }

    data_daily = call_api(params_daily, api_key, ticker)

    if data_daily is None or "Time Series (Daily)" not in data_daily:
        print(f"   ❌ Failed to fetch data for {ticker}")
        failed_tickers.append(ticker)
        continue

    df = pd.DataFrame.from_dict(data_daily["Time Series (Daily)"], orient="index")
    df = df.rename_axis("Date").reset_index()
    df["Date"] = pd.to_datetime(df["Date"])
    df["Ticker"] = ticker

    df = df.rename(columns={
        "1. open": "Open",
        "2. high": "High",
        "3. low": "Low",
        "4. close": "Close",
        "5. volume": "Volume"
    })

    numeric_cols = ["Open", "High", "Low", "Close", "Volume"]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df.sort_values("Date").reset_index(drop=True)

    print("   📈 Calculating SMA (20)...")
    df["SMA_20"] = calculate_sma(df["Close"], period=20)

    print("   📈 Calculating RSI (14)...")
    df["RSI_14"] = calculate_rsi(df["Close"], period=14)

    print("   📈 Calculating MACD...")
    df["MACD"], df["MACD_Signal"], df["MACD_Hist"] = calculate_macd(df["Close"])

    all_data.append(df)
    successful_tickers.append(ticker)
    print(f"   ✅ Successfully processed {ticker} ({len(df):,} rows)")


# ===========================
# SAVE FINAL DATASET
# ===========================

print(f"\n{'='*60}")
print("💾 Saving Results...")
print(f"{'='*60}")

if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    final_df = final_df.sort_values(["Ticker", "Date"])

    column_order = [
        "Ticker", "Date", "Open", "High", "Low", "Close", "Volume",
        "SMA_20", "RSI_14", "MACD", "MACD_Signal", "MACD_Hist"
    ]
    final_df = final_df[column_order]

    output_file = "alpha_vantage_dataset.csv"
    final_df.to_csv(output_file, index=False)

    print(f"\n🎉 SUCCESS!")
    print(f"{'='*60}")
    print(f"✅ Saved: {output_file}")
    print(f"✅ Total rows: {len(final_df):,}")
    print(f"✅ Tickers processed: {', '.join(successful_tickers)}")
    print(f"✅ Date range: {final_df['Date'].min().date()} to {final_df['Date'].max().date()}")
    
    if failed_tickers:
        print(f"\n⚠️ Failed tickers: {', '.join(failed_tickers)}")

else:
    print("\n❌ No data was successfully fetched.")


📊 Alpha Vantage Multi-Key Stock Data Fetcher
✅ 5 tickers with 5 dedicated API keys
✅ Each ticker uses its own API key


[1/5] Processing: AAPL
   🔑 Using API key: 9VNG...YZMJ
   📋 Response keys: ['Meta Data', 'Time Series (Daily)']
   ✅ Successfully received data!
   📈 Calculating SMA (20)...
   📈 Calculating RSI (14)...
   📈 Calculating MACD...
   ✅ Successfully processed AAPL (6,554 rows)

[2/5] Processing: MSFT
⏳ Waiting 12 seconds...
   🔑 Using API key: UVAA...7AGC
   📋 Response keys: ['Meta Data', 'Time Series (Daily)']
   ✅ Successfully received data!
   📈 Calculating SMA (20)...
   📈 Calculating RSI (14)...
   📈 Calculating MACD...
   ✅ Successfully processed MSFT (6,554 rows)

[3/5] Processing: GOOG
⏳ Waiting 12 seconds...
   🔑 Using API key: 63BQ...PB9X
   📋 Response keys: ['Meta Data', 'Time Series (Daily)']
   ✅ Successfully received data!
   📈 Calculating SMA (20)...
   📈 Calculating RSI (14)...
   📈 Calculating MACD...
   ✅ Successfully processed GOOG (2,932 rows)

[4/5] 

#  Dataset 4: Fama-French Factor Data

In [11]:
import pandas as pd
import urllib.request
import zipfile
import io

url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip"
urllib.request.urlretrieve(url, "ff3.zip")

with zipfile.ZipFile("ff3.zip", "r") as z:
    z.extractall()



TypeError: unsupported operand type(s) for /: 'str' and 'float'

In [19]:
import pandas as pd

# ----------------------------
# Load your dataset
# ----------------------------
df = pd.read_csv("F-F_Research_Data_Factors.csv")

# ----------------------------
# Convert YYYYMM → datetime
# ----------------------------
date_col = df.columns[0]      # First column is the date
df[date_col] = pd.to_datetime(df[date_col].astype(str), format="%Y%m")

# ----------------------------
# Fill NaN values
# ----------------------------
df = df.ffill().bfill()       # Forward fill → backward fill

# ----------------------------
# Show results
# ----------------------------
df
df.to_csv('F-F_Research_Data_Factors.csv')


# Dataset 5: CoinGecko Cryptocurrency Data API

In [1]:
# ==========================================
# CoinGecko Crypto Data Loader + Concatenate All Coins
# Fetches past 365 days of historical price, market cap, and 24h volume
# Handles multiple coins and fiat currencies
# Concatenates all coins into a single multi-index DataFrame
# ==========================================

import pandas as pd
from pycoingecko import CoinGeckoAPI
import time

class CryptoDataLoader:
    def __init__(self, coins=None, vs_currencies=['usd'], top_n=10):
        """
        coins: list of coin IDs, e.g., ['bitcoin','ethereum']
        vs_currencies: list of fiat currencies, e.g., ['usd','eur']
        top_n: number of top coins by market cap if coins is None
        """
        self.cg = CoinGeckoAPI()
        self.coins = coins
        self.vs_currencies = vs_currencies
        self.top_n = top_n

    def get_top_coins(self):
        """Get top N coins by market cap."""
        markets = self.cg.get_coins_markets(vs_currency='usd', per_page=self.top_n, page=1)
        return [m['id'] for m in markets]

    def fetch_historical(self, coin_id, vs_currency='usd', days=365):
        """Fetch past N days of historical data for one coin in one currency."""
        data = self.cg.get_coin_market_chart_by_id(
            id=coin_id,
            vs_currency=vs_currency,
            days=days  # free tier limit
        )
        # Convert to DataFrame
        df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
        df['market_cap'] = [x[1] for x in data['market_caps']]
        df['volume_24h'] = [x[1] for x in data['total_volumes']]
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        return df

    def load(self):
        """Fetch historical data for all coins and currencies."""
        if self.coins is None:
            self.coins = self.get_top_coins()
        all_data = []
        for coin in self.coins:
            coin_frames = []
            for currency in self.vs_currencies:
                print(f"Fetching {coin} in {currency}...")
                df = self.fetch_historical(coin, currency)
                # Prefix columns with currency
                df = df.add_prefix(f"{currency}_")
                coin_frames.append(df)
                time.sleep(1)  # avoid hitting rate limits
            # Combine multiple currencies horizontally
            combined = pd.concat(coin_frames, axis=1)
            # Add coin ID as a column for concatenation
            combined['coin'] = coin
            all_data.append(combined)
        # Concatenate all coins into a single DataFrame
        final_df = pd.concat(all_data)
        # Set multi-index: coin -> timestamp
        final_df.set_index('coin', append=True, inplace=True)
        final_df = final_df.reorder_levels(['coin', final_df.index.names[0]])
        final_df.sort_index(inplace=True)
        return final_df

# ==========================================
# Example usage
# ==========================================
if __name__ == "__main__":
    loader = CryptoDataLoader(coins=['bitcoin', 'ethereum'], vs_currencies=['usd', 'eur'])
    df_all = loader.load()

    # Show top rows
    print(df_all.head())

    # Save concatenated DataFrame to CSV
    df_all.to_csv("crypto_365days_concatenated.csv")
    print("Saved crypto_365days_concatenated.csv")


Fetching bitcoin in usd...
Fetching bitcoin in eur...
Fetching ethereum in usd...
Fetching ethereum in eur...
                       usd_price  usd_market_cap  usd_volume_24h  \
coin    timestamp                                                  
bitcoin 2024-11-21  94217.022296    1.862619e+12    8.074727e+10   
        2024-11-22  98509.118591    1.948285e+12    1.181639e+11   
        2024-11-23  98927.494946    1.958091e+12    8.574617e+10   
        2024-11-24  97679.463816    1.931256e+12    4.741420e+10   
        2024-11-25  98015.935529    1.939446e+12    5.066568e+10   

                       eur_price  eur_market_cap  eur_volume_24h  
coin    timestamp                                                 
bitcoin 2024-11-21  89326.405103    1.766102e+12    7.655584e+10  
        2024-11-22  94069.115598    1.861206e+12    1.128380e+11  
        2024-11-23  94955.556023    1.879474e+12    8.230347e+10  
        2024-11-24  93760.270690    1.853768e+12    4.551180e+10  
        202

# Dataset 6: ETF & Mutual Fund Database

In [4]:
# ==========================================
# Exchange Rate Data Preprocessing Script
# Loads CSV, cleans headers, converts to numeric, fills NaNs, and saves cleaned CSV
# ==========================================

import pandas as pd

# --------------------------
# Step 1: Load CSV
# --------------------------
df = pd.read_csv("Exchange_Rate_Report.csv", index_col=0)

# Strip extra spaces from column names
df.columns = df.columns.str.strip()

# Convert all columns to numeric (non-numeric -> NaN)
df = df.apply(pd.to_numeric, errors='coerce')

# Convert index to datetime
df.index = pd.to_datetime(df.index, errors='coerce')

# --------------------------
# Step 2: Fill NaN values
# --------------------------
# 1. Forward fill missing values
df_filled = df.fillna(method='ffill')

# 2. Backward fill remaining missing values
df_filled = df_filled.fillna(method='bfill')

# 3. Linear interpolation for any remaining small gaps
df_filled = df_filled.interpolate(method='linear')

# --------------------------
# Step 3: Verify
# --------------------------
print("First 5 rows of cleaned data:")
print(df_filled.head())

print("\nNumber of remaining NaNs (should be 0):")
print(df_filled.isna().sum().sum())

# --------------------------
# Step 4: Save cleaned CSV
# --------------------------
df_filled.to_csv("Exchange_Rate_Report_Cleaned.csv")
print("\nCleaned CSV saved as 'Exchange_Rate_Report_Cleaned.csv'")


C:\Users\itsam\AppData\Local\Temp\ipykernel_13168\3958645185.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled = df.fillna(method='ffill')
C:\Users\itsam\AppData\Local\Temp\ipykernel_13168\3958645185.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled = df_filled.fillna(method='bfill')


First 5 rows of cleaned data:
            Algerian dinar   (DZD)  Australian dollar   (AUD)  \
Date                                                            
1994-01-03                 74.9996                     0.6873   
1994-01-04                 74.9996                     0.6873   
1994-01-05                 74.9996                     0.6870   
1994-01-06                 74.9996                     0.6857   
1994-01-07                 74.9996                     0.6857   

            Austrian schilling   (ATS)  Belgian franc   (BEF)  \
Date                                                            
1994-01-03                      12.194                 32.025   
1994-01-04                      12.234                 32.025   
1994-01-05                      12.187                 32.025   
1994-01-06                      12.187                 32.025   
1994-01-07                      12.235                 32.025   

            Botswana pula   (BWP)  Brazilian real   (BRL) 

# Dataset 7: FRED Economic Indicators (for portfolio context

In [5]:
# ==========================================
# FRED Economic Indicators Loader
# ==========================================

import pandas as pd
from fredapi import Fred

# --------------------------
# Step 1: Initialize FRED API
# --------------------------
api_key = '7242fe69a51188cad3114480ea35cf15'
fred = Fred(api_key=api_key)

# --------------------------
# Step 2: Define FRED series IDs
# --------------------------
series_ids = {
    "3mo_treasury_yield": "DGS3MO",
    "2yr_treasury_yield": "DGS2",
    "10yr_treasury_yield": "DGS10",
    "30yr_treasury_yield": "DGS30",
    "unemployment_rate": "UNRATE",
    "pce_inflation": "PCEPI",
    "vix": "VIXCLS"
}

# --------------------------
# Step 3: Fetch data
# --------------------------
data_frames = []

for name, series_id in series_ids.items():
    print(f"Fetching {series_id} ({name})...")
    df = fred.get_series(series_id)
    df = df.to_frame(name)   # convert to DataFrame with column name
    data_frames.append(df)

# --------------------------
# Step 4: Combine all series into one DataFrame
# --------------------------
df_all = pd.concat(data_frames, axis=1)

# --------------------------
# Step 5: Handle missing data
# --------------------------
# Forward fill, backward fill, interpolate if needed
df_all = df_all.fillna(method='ffill')
df_all = df_all.fillna(method='bfill')
df_all = df_all.interpolate(method='linear')

# --------------------------
# Step 6: Verify
# --------------------------
print("First 5 rows of combined FRED indicators:")
print(df_all.head())

print("\nNumber of remaining NaNs (should be 0):", df_all.isna().sum().sum())

# --------------------------
# Step 7: Save to CSV
# --------------------------
df_all.to_csv("FRED_Economic_Indicators.csv")
print("\nSaved combined FRED data as 'FRED_Economic_Indicators.csv'")


Fetching DGS3MO (3mo_treasury_yield)...
Fetching DGS2 (2yr_treasury_yield)...
Fetching DGS10 (10yr_treasury_yield)...
Fetching DGS30 (30yr_treasury_yield)...
Fetching UNRATE (unemployment_rate)...
Fetching PCEPI (pce_inflation)...
Fetching VIXCLS (vix)...
First 5 rows of combined FRED indicators:
            3mo_treasury_yield  2yr_treasury_yield  10yr_treasury_yield  \
1948-01-01               17.01                7.26                 4.06   
1948-02-01               17.01                7.26                 4.06   
1948-03-01               17.01                7.26                 4.06   
1948-04-01               17.01                7.26                 4.06   
1948-05-01               17.01                7.26                 4.06   

            30yr_treasury_yield  unemployment_rate  pce_inflation    vix  
1948-01-01                  7.7                3.4         15.164  17.24  
1948-02-01                  7.7                3.8         15.164  17.24  
1948-03-01                

C:\Users\itsam\AppData\Local\Temp\ipykernel_13168\1073717842.py:47: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_all = df_all.fillna(method='ffill')
C:\Users\itsam\AppData\Local\Temp\ipykernel_13168\1073717842.py:48: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_all = df_all.fillna(method='bfill')


# 3. TAX OPTIMIZATION AGENT DATASETS

In [6]:
# ==========================================
# IRS Tax Forms Downloader for Tax Optimization
# Downloads all relevant IRS forms for analysis
# ==========================================

import os
import requests
from bs4 import BeautifulSoup

# --------------------------
# Step 1: Setup directories
# --------------------------
year = 2024
base_dir = f"IRS_Forms_{year}"
os.makedirs(base_dir, exist_ok=True)

# --------------------------
# Step 2: List of target forms
# --------------------------
# Core forms relevant for tax optimization
forms_to_download = [
    "1040",      # Individual income
    "1040A",     # Simplified (if available)
    "1040EZ",    # Simplified (if available)
    "1040SR",    # Senior
    "Schedule A", # Itemized deductions
    "Schedule B", # Interest/dividends
    "Schedule C", # Business profit/loss
    "Schedule D", # Capital gains/losses
    "Schedule E", # Rental income
    "Schedule SE", # Self-employment tax
    "Form 1099",  # Various 1099s
    "Form W-2",   # Wage reporting
    "Form 4562",  # Depreciation & amortization
    "Form 8829",  # Home office deduction
    "Form 8949",  # Sales & other dispositions of capital assets
]

# --------------------------
# Step 3: IRS forms base URL
# --------------------------
irs_base_pdf_url = "https://www.irs.gov/pub/irs-pdf/"

# Map form name to actual PDF file names (approximate; IRS naming conventions)
form_pdf_map = {
    "1040": f"f1040.pdf",
    "Schedule A": f"f1040sa.pdf",
    "Schedule B": f"f1040sb.pdf",
    "Schedule C": f"f1040sc.pdf",
    "Schedule D": f"f1040sd.pdf",
    "Schedule E": f"f1040se.pdf",
    "Schedule SE": f"f1040sese.pdf",
    "Form 4562": f"f4562.pdf",
    "Form 8829": f"f8829.pdf",
    "Form 8949": f"f8949.pdf",
    # Add more mappings as needed
}

# --------------------------
# Step 4: Download forms
# --------------------------
for form in forms_to_download:
    pdf_file = form_pdf_map.get(form)
    if not pdf_file:
        print(f"Skipping {form} (no mapping found)")
        continue

    url = irs_base_pdf_url + pdf_file
    print(f"Downloading {form} from {url}...")

    try:
        response = requests.get(url)
        if response.status_code == 200:
            save_path = os.path.join(base_dir, pdf_file)
            with open(save_path, "wb") as f:
                f.write(response.content)
            print(f"Saved {pdf_file}")
        else:
            print(f"Failed to download {form}: Status code {response.status_code}")
    except Exception as e:
        print(f"Error downloading {form}: {e}")

print("\nAll downloads completed!")


Saved f1040.pdf
Skipping 1040A (no mapping found)
Skipping 1040EZ (no mapping found)
Skipping 1040SR (no mapping found)
Saved f1040sa.pdf
Saved f1040sb.pdf
Saved f1040sc.pdf
Saved f1040sd.pdf
Saved f1040se.pdf
Failed to download Schedule SE: Status code 404
Skipping Form 1099 (no mapping found)
Skipping Form W-2 (no mapping found)
Saved f4562.pdf
Saved f8829.pdf
Saved f8949.pdf

All downloads completed!


In [7]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# -----------------------------
# Setup
# -----------------------------
BASE_URL = "https://www.irs.gov/statistics"
SAVE_DIR = "IRS_Tax_Statistics"
os.makedirs(SAVE_DIR, exist_ok=True)

# File types to download
FILE_TYPES = (".pdf", ".xls", ".xlsx", ".csv")

# -----------------------------
# Helper: download a file
# -----------------------------
def download_file(url, folder=SAVE_DIR):
    filename = url.split("/")[-1]
    filepath = os.path.join(folder, filename)

    try:
        print(f"Downloading: {url}")
        r = requests.get(url, timeout=20)
        if r.status_code == 200:
            with open(filepath, "wb") as f:
                f.write(r.content)
            print(f"Saved → {filename}")
        else:
            print(f"Failed ({r.status_code}): {url}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")

# -----------------------------
# Step 1: Scrape IRS Statistics Main Page
# -----------------------------
print("Fetching IRS Statistics main page...")
response = requests.get(BASE_URL)
soup = BeautifulSoup(response.text, "html.parser")

# Collect all sub-pages under /statistics/
subpages = []

for a in soup.find_all("a", href=True):
    href = a["href"]
    if href.startswith("/statistics/"):
        full_url = urljoin(BASE_URL, href)
        subpages.append(full_url)

subpages = list(set(subpages))  # Remove duplicates
print(f"Found {len(subpages)} subpages.")

# -----------------------------
# Step 2: For each subpage, download files
# -----------------------------
for page in subpages:
    print(f"\n📄 Checking page: {page}")
    try:
        res = requests.get(page, timeout=20)
        sp = BeautifulSoup(res.text, "html.parser")

        for link in sp.find_all("a", href=True):
            file_url = link["href"]
            if file_url.endswith(FILE_TYPES):
                full_url = urljoin(page, file_url)
                download_file(full_url)

    except Exception as e:
        print(f"Error processing {page}: {e}")

print("\n🎉 All IRS tax statistics downloaded!")


Fetching IRS Statistics main page...
Found 39 subpages.

📄 Checking page: https://www.irs.gov/statistics/soi-tax-stats-historical-data-tables
Downloading: https://www.irs.gov/pub/irs-soi/histab21c.xlsx
Saved → histab21c.xlsx
Downloading: https://www.irs.gov/pub/irs-soi/histab21d.xlsx
Saved → histab21d.xlsx
Downloading: https://www.irs.gov/pub/irs-soi/histab21e.xlsx
Saved → histab21e.xlsx
Downloading: https://www.irs.gov/pub/irs-soi/histab21f.xlsx
Saved → histab21f.xlsx

📄 Checking page: https://www.irs.gov/statistics/credits-and-deductions-gap
Downloading: https://www.irs.gov/pub/irs-soi/24rpcreditdeductiongapsummary.pdf
Saved → 24rpcreditdeductiongapsummary.pdf
Downloading: https://www.irs.gov/pub/irs-soi/24rpeitcunderclaims.pdf
Saved → 24rpeitcunderclaims.pdf
Downloading: https://www.irs.gov/pub/irs-soi/24rpctcunderclaims.pdf
Saved → 24rpctcunderclaims.pdf
Downloading: https://www.irs.gov/pub/irs-soi/24rpedtcunderclaims.pdf
Saved → 24rpedtcunderclaims.pdf
Downloading: https://www.irs


KeyboardInterrupt



#  4. COORDINATOR AGENT DATASETS

In [14]:
import requests
import pdfplumber
from bs4 import BeautifulSoup
import json
import os
import re
import time

# -----------------------------
# Setup directories
# -----------------------------
os.makedirs("raw_data", exist_ok=True)
os.makedirs("structured_data", exist_ok=True)

# -----------------------------
# Step 1: Download Bogleheads PDF
# -----------------------------
bogle_pdf_url = "https://boglecenter.net/wp-content/uploads/Bogleheads-University-Full-Slide-Deck-1.pdf"
bogle_pdf_path = os.path.join("raw_data", "bogle_slides.pdf")

r = requests.get(bogle_pdf_url)
with open(bogle_pdf_path, "wb") as f:
    f.write(r.content)
print("Downloaded Bogleheads PDF:", bogle_pdf_path)

# -----------------------------
# Step 2: Extract text from PDF
# -----------------------------
bogle_text = ""
with pdfplumber.open(bogle_pdf_path) as pdf:
    for page in pdf.pages:
        bogle_text += page.extract_text() + "\n"

with open(os.path.join("raw_data", "bogle_slides.txt"), "w", encoding="utf-8") as f:
    f.write(bogle_text)
print("Extracted text from PDF.")

# -----------------------------
# Step 3: Scrape Investopedia articles
# -----------------------------
investopedia_urls = [
    "https://www.investopedia.com/retirement-planning-strategies-5184216",
    "https://www.investopedia.com/saving-for-college-5183900"
]

investopedia_texts = []
for url in investopedia_urls:
    try:
        resp = requests.get(url, timeout=10)
        soup = BeautifulSoup(resp.text, "html.parser")
        paragraphs = soup.find_all("p")
        text = " ".join(p.get_text() for p in paragraphs)
        investopedia_texts.append({"url": url, "text": text})
        time.sleep(1)  # polite scraping
    except Exception as e:
        print(f"Error scraping {url}: {e}")

# Save raw text
with open(os.path.join("raw_data", "investopedia_articles.json"), "w", encoding="utf-8") as f:
    json.dump(investopedia_texts, f, indent=2)

# -----------------------------
# Step 4: Parse goal examples (heuristic)
# -----------------------------
goal_examples = []

# Example: extract SMART goal pattern from Bogleheads PDF text
pattern1 = re.compile(r'“?We will be worth \$([\d,\.]+) by (.+?)”', re.IGNORECASE)
for idx, match in enumerate(pattern1.finditer(bogle_text)):
    goal_examples.append({
        "id": f"bogle_{idx}",
        "source": "Bogleheads PDF",
        "goal_type": "Net Worth Target",
        "target_amount": float(match.group(1).replace(",", "")),
        "target_date": match.group(2),
        "allocation": None,
        "risk_level": None,
        "obstacles": []
    })

# Simple heuristic: look for "retire", "college", "home" in Investopedia articles
for art in investopedia_texts:
    sentences = re.split(r'\.|\n', art["text"])
    for idx, s in enumerate(sentences):
        if any(word in s.lower() for word in ["retire", "college", "home", "education"]):
            goal_examples.append({
                "id": f"investopedia_{idx}",
                "source": art["url"],
                "goal_type": s.strip()[:50],  # first 50 chars as heuristic title
                "target_amount": None,
                "target_date": None,
                "allocation": None,
                "risk_level": None,
                "obstacles": []
            })

# -----------------------------
# Step 5: Save structured JSON (RAG-ready)
# -----------------------------
structured_file = os.path.join("structured_data", "financial_goals.jsonl")
with open(structured_file, "w", encoding="utf-8") as f:
    for goal in goal_examples:
        f.write(json.dumps(goal) + "\n")

print(f"✅ Scraping & extraction done. Total goals: {len(goal_examples)}")
print(f"Saved structured dataset to {structured_file}")


Downloaded Bogleheads PDF: raw_data\bogle_slides.pdf
Extracted text from PDF.
✅ Scraping & extraction done. Total goals: 0
Saved structured dataset to structured_data\financial_goals.jsonl


In [15]:
import requests
from bs4 import BeautifulSoup
import json
import re
import os
import time

# Directories for raw and structured data
os.makedirs("case_study_raw", exist_ok=True)
os.makedirs("case_study_structured", exist_ok=True)

# -----------------------------
# Step 1: List of URLs to scrape
# -----------------------------
urls = [
    "https://www.investopedia.com/articles/pf/08/investing-case-study.asp",
    "https://www.morningstar.com/articles/1023454/sample-investment-case-study"
    # Add more URLs as needed
]

# -----------------------------
# Step 2: Scrape text from each page
# -----------------------------
case_studies = []
for url in urls:
    try:
        resp = requests.get(url, timeout=10)
        soup = BeautifulSoup(resp.text, "html.parser")
        paragraphs = soup.find_all("p")
        text = " ".join(p.get_text() for p in paragraphs)
        case_studies.append({"url": url, "text": text})
        time.sleep(1)  # be polite
    except Exception as e:
        print(f"Error scraping {url}: {e}")

# Save raw scraped text
with open(os.path.join("case_study_raw", "scraped_articles.json"), "w", encoding="utf-8") as f:
    json.dump(case_studies, f, indent=2)

# -----------------------------
# Step 3: Heuristic parsing into structured fields
# -----------------------------
structured_cases = []

for idx, case in enumerate(case_studies):
    text = case["text"]

    # Example heuristic splitting (can be improved or use NLP)
    sentences = re.split(r'\.|\n', text)
    initial_situation = sentences[0] if len(sentences) > 0 else ""
    decisions_made = sentences[1] if len(sentences) > 1 else ""
    alternatives = sentences[2] if len(sentences) > 2 else ""
    outcomes = sentences[3] if len(sentences) > 3 else ""
    lessons = sentences[4] if len(sentences) > 4 else ""

    structured_cases.append({
        "id": f"case_{idx}",
        "source_url": case["url"],
        "initial_situation": initial_situation.strip(),
        "decisions_made": decisions_made.strip(),
        "alternatives_considered": alternatives.strip(),
        "outcome": outcomes.strip(),
        "lessons_learned": lessons.strip()
    })

# -----------------------------
# Step 4: Save structured dataset
# -----------------------------
out_file = os.path.join("case_study_structured", "financial_case_studies.jsonl")
with open(out_file, "w", encoding="utf-8") as f:
    for case in structured_cases:
        f.write(json.dumps(case) + "\n")

print(f"✅ Scraping complete. {len(structured_cases)} cases saved to {out_file}")


✅ Scraping complete. 2 cases saved to case_study_structured\financial_case_studies.jsonl


In [16]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_json("hf://datasets/Akhil-Theerthala/PersonalFinance_v2/finance_cotr.jsonl", lines=True)

In [18]:
df.to_csv('finance dataset.csv')

# State-Individual-Income-Tax-Rates-and-Brackets-2015-2024_Tax_Foundation

In [3]:
import pandas as pd
import re

file_path = "C:\\Users\\itsam\\Downloads\\agents datset\\TAX OPTIMIZATION AGENT DATASETS\\State-Individual-Income-Tax-Rates-and-Brackets-2015-2024_Tax_Foundation.xlsx"

# Load all sheets
sheets = pd.read_excel(file_path, sheet_name=None)

cleaned_frames = []

# -----------------------------
# 1. Function — Extract footnotes from the bottom
# -----------------------------

def extract_footnotes(text_block):
    """
    Extracts footnotes like:
        (a) explanation...
        (b) explanation...
    Returns dictionary:
        {'a': 'Local income taxes...', 'b': 'These states allow...', ...}
    """

    footnote_pattern = r"\(([a-z])\)\s*(.*?)\s*(?=\([a-z]\)|$)"

    notes = dict(re.findall(footnote_pattern, text_block, flags=re.S))
    return notes


# -----------------------------
# 2. Main sheet processor
# -----------------------------

def process_sheet(df):
    df = df.copy()

    # Separate data rows vs informational text
    footer_rows = df[df.apply(lambda r: r.astype(str).str.contains(r"\([a-z]\)").any(), axis=1)]

    # Join footer rows into one long text block
    footer_text = " ".join(" ".join(row.astype(str).values) for idx, row in footer_rows.iterrows())

    # Extract footnote mapping
    footnotes = extract_footnotes(footer_text)

    # Drop the footer rows from main data
    df = df.drop(footer_rows.index)

    # -----------------------------
    # Extract footnote letters from state column
    # -----------------------------

    if "State" not in df.columns:
        # Try to find state column by heuristic
        for col in df.columns:
            if df[col].astype(str).str.contains(r"Ala\.|Alaska|N\.Y\.").any():
                df.rename(columns={col: "State"}, inplace=True)
                break

    # Find things like "(a, b, c)"
    df["footnotes"] = df["State"].astype(str).str.extract(r"\((.*?)\)", expand=False)

    # Remove "(a, b, c)" from State name
    df["State"] = df["State"].astype(str).str.replace(r"\(.*?\)", "", regex=True).str.strip()

    # -----------------------------
    # Expand footnotes into columns
    # -----------------------------

    for code, explanation in footnotes.items():
        col_name = f"note_{code}"
        df[col_name] = df["footnotes"].apply(
            lambda x: explanation if (isinstance(x, str) and code in x) else None
        )

    return df


# -----------------------------
# 3. Apply to all sheets
# -----------------------------

for sheet_name, df in sheets.items():
    print(f"Processing sheet: {sheet_name}")
    cleaned = process_sheet(df)
    cleaned["source_sheet"] = sheet_name
    cleaned_frames.append(cleaned)

combined_df = pd.concat(cleaned_frames, ignore_index=True)

combined_df.to_csv("cleaned_tax_with_notes.csv", index=False)

print("Scleaned_tax_with_notes.csv")


Processing sheet: 2024
Processing sheet: 2023
Processing sheet: 2022
Processing sheet: 2021
Processing sheet: 2020
Processing sheet: 2019
Processing sheet: 2018
Processing sheet: 2017
Processing sheet: 2016
Processing sheet: 2015
Scleaned_tax_with_notes.csv


In [4]:
import pandas as pd
import re

# ---------------------------------------
# 1. Load the saved cleaned CSV file
# ---------------------------------------
file_path = "cleaned_tax_with_notes.csv"

df = pd.read_csv(file_path)

# -----------------------------------------------------
# 2. Remove informational/explanatory text from columns
# -----------------------------------------------------

def remove_info_text(cell):
    """
    Removes long explanatory text or footnotes from any cell.
    Rules:
    - Remove text with long sentences
    - Remove extra symbols
    """
    if pd.isna(cell):
        return cell
    
    text = str(cell)

    # Remove long multi-sentence information
    if len(text.split()) > 25:  # too long => probably explanation
        return None
    
    # Remove footnote patterns like (a), (b, c)
    text = re.sub(r"\(.*?\)", "", text)

    # Remove excessive hyphens, quotes, URLs, and reference text
    text = re.sub(r"http[s]?:\/\/\S+", "", text)
    text = re.sub(r"[\"“”]", "", text)

    return text.strip()


df = df.applymap(remove_info_text)

# -----------------------------------------------------
# 3. Drop any remaining rows that are all NaN
# -----------------------------------------------------
df = df.dropna(how='all')

# -----------------------------------------------------
# 4. Drop rows where essential columns became empty
# -----------------------------------------------------
if "State" in df.columns:
    df = df[df["State"].notna()]

# -----------------------------------------------------
# 5. Save cleaned final dataset
# -----------------------------------------------------
output_path = "final_tax_cleaned.csv"
df.to_csv(output_path, index=False)

print("Final cleaned file saved to:", output_path)


C:\Users\itsam\AppData\Local\Temp\ipykernel_8004\3936340586.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(remove_info_text)


Final cleaned file saved to: final_tax_cleaned.csv


In [1]:
import pandas as pd
import os
from glob import glob
import re

# Path to your folder containing CSV files
folder_path = "C:\\Users\\itsam\\Downloads\\archive"  # update as needed

# Get a list of all CSV files in the folder
csv_files = glob(os.path.join(folder_path, "*.csv"))

# Initialize a list to store dataframes
dfs = []

# Loop through files and read them
for file in csv_files:
    try:
        df = pd.read_csv(file)
        
        # Extract the first 4-digit number in filename as year
        match = re.search(r'\b(20\d{2}|19\d{2})\b', os.path.basename(file))
        if match:
            year = int(match.group(0))
        else:
            year = None  # If no year found, leave as None
        
        df['year'] = year
        dfs.append(df)
    
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Concatenate all dataframes
if dfs:
    final_df = pd.concat(dfs, ignore_index=True)
    # Save to a single CSV
    final_df.to_csv(os.path.join(folder_path, "combined_data.csv"), index=False)
    print("All CSV files have been combined into 'combined_data.csv'!")
else:
    print("No CSV files were read successfully.")


C:\Users\itsam\AppData\Local\Temp\ipykernel_17156\3138123710.py:18: DtypeWarning: Columns (38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


All CSV files have been combined into 'combined_data.csv'!


In [3]:
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import os

# Initialize API
api = KaggleApi()
api.authenticate()

# Dataset info
dataset = 'irs/irs-990'
download_path = './irs-990.zip'   # where zip will be saved
extract_path = './irs-990'        # folder where files will be extracted

# Download dataset
api.dataset_download_files(dataset, path='.', unzip=False)

# Extract zip
with zipfile.ZipFile(download_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Dataset downloaded and extracted to {extract_path}")


Could not find kaggle.json. Make sure it's located in C:\Users\itsam\.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


NameError: name 'exit' is not defined

In [1]:
print("They're both strings/string literals.")

They're both strings/string literals.
